<a href="https://colab.research.google.com/github/vlamen/tue-deeplearning/blob/main/assignments/assignment_4/assignment_4_skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group Number:

# Student 1:

# Student 2:

# Student 3:

# Imports

In [4]:
import numpy as np
import pickle
import requests
import torch

# other imports go here

# Data loading and inspection

In [5]:
# load and inspect data
data_location = 'https://surfdrive.surf.nl/files/index.php/s/K3ArFDQJb5USQ6K/download'
data_request = requests.get(data_location)
full_data = pickle.loads(data_request.content)

# Data augmentation and pipeline

In [6]:
# code for data augmentation pipeline 
labeled_data_full = full_data['labeled_data']
train_data = labeled_data_full['data']
train_labels = labeled_data_full['labels']
unlabeled_data = full_data['unlabeled_data']
train_data.shape
full_data.keys()

dict_keys(['unlabeled_data', 'labeled_data', 'representative_set_1', 'representative_set_2'])

In [47]:
from torch.utils.data import Dataset, DataLoader,TensorDataset
from torch.nn.functional import normalize
class MyDataset(Dataset):
    def __init__(self, data, labels = None):
        self.data = torch.FloatTensor(data)
        if labels is not None:
            self.labels = torch.FloatTensor(labels)
        else:
            self.labels = None

    def __getitem__(self, index):
        x = self.data[index]  
        x = x / 255 # normalize
        if self.labels is not None:
            y = self.labels[index] # .view(1,2,5)
        else:
            y = np.array([])

        return x, y
    def __len__(self):
        return len(self.data)

In [48]:
train_dataset=MyDataset(unlabeled_data)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100)
for x, y in train_loader:
    print(x)
    print(x.shape)
    break

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.

# Model definitions

In [49]:
# code for model definitions goes here
import torch
import torch.nn as nn
import torch.optim as optim

latent_dim = 5
x_dim = 32*32 # should be 32 32 but testing it on 28 28 atm
hidden_dim = 500

lr = 1e-3
epochs = 20


class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.fc_input = nn.Linear(input_dim, hidden_dim)
        self.fc_hidden = nn.Linear(hidden_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_sigma = nn.Linear (hidden_dim, latent_dim)
        
    def forward(self, x):
        h = torch.relu(self.fc_input(x))
        h = torch.relu(self.fc_hidden(h))
        mu = self.fc_mu(h)
        log_sigma = self.fc_sigma(h)
        z = self.reparameterization(mu, log_sigma)

        return z, mu, log_sigma
    
    def reparameterization(self, mu, log_sigma):
        sigma = torch.exp(log_sigma)
        epsilon = torch.randn_like(sigma)
        z = mu + sigma * epsilon
        
        return z


class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc_hidden1 = nn.Linear(latent_dim, hidden_dim)
        self.fc_hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_output = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        h = torch.relu(self.fc_hidden1(x))
        h = torch.relu(self.fc_hidden2(h))
        x_reconstr = torch.sigmoid(self.fc_output(h))
        return x_reconstr


class VAE(nn.Module):
    def __init__(self, encoder, decoder):
        super(VAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
                
    def forward(self, x):
        z, mu, log_sigma = self.encoder(x)
        x_reconstr = self.decoder(z)
        
        return x_reconstr, mu, log_sigma


# cuda = True  # NOTE: if running in Google Colab, make sure to go to "Edit > Notebook settings" and set "Hardware accelerator" to "GPU"
# DEVICE = torch.device("cuda" if cuda else "cpu")

encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim=hidden_dim, output_dim=x_dim)

vae = VAE(encoder=encoder, decoder=decoder) #.to(DEVICE)



def loss_function(x, x_reconstr, mu, log_sigma):
    reconstr_loss = nn.functional.mse_loss(x_reconstr, x, reduction='sum')
    kl_loss = 0.5 * torch.sum(mu.pow(2) + (2*log_sigma).exp() - 2*log_sigma - 1)
    total_loss = reconstr_loss + kl_loss
    return total_loss, reconstr_loss, kl_loss

optimizer = optim.Adam(vae.parameters(), lr=lr)

print("Start training VAE...")
vae.train()

for epoch in range(epochs):
    overall_loss = 0
    overall_reconstr_loss = 0
    overall_kl_loss = 0
    for batch_idx, (x, _) in enumerate(train_loader):
        x = x.view(x.shape[0], x_dim)
        x = x #.to(DEVICE)
        optimizer.zero_grad()

        x_reconstr, mu, log_sigma = vae(x)
        loss, reconstr_loss, kl_loss = loss_function(x, x_reconstr, mu, log_sigma)
        
        overall_loss += loss.item()
        overall_reconstr_loss += reconstr_loss.item()
        overall_kl_loss += kl_loss.item()
        
        loss.backward()
        optimizer.step()
        
    n_datapoints = batch_idx * 100
    print("\tEpoch", epoch + 1, "\tAverage Loss: ", overall_loss / n_datapoints, "\tReconstruction Loss:", overall_reconstr_loss / n_datapoints, "\tKL Loss:", overall_kl_loss / n_datapoints)
    
print("Training complete!")

Start training VAE...
	Epoch 1 	Average Loss:  45.37398701435811 	Reconstruction Loss: 41.462913615694376 	KL Loss: 3.9110733421796997
	Epoch 2 	Average Loss:  28.301915155948357 	Reconstruction Loss: 23.17098951420729 	KL Loss: 5.130925645275926
	Epoch 3 	Average Loss:  26.68729455425012 	Reconstruction Loss: 21.33390914858078 	KL Loss: 5.353385449265882
	Epoch 4 	Average Loss:  25.8711979009562 	Reconstruction Loss: 20.316018910058215 	KL Loss: 5.555178967332288
	Epoch 5 	Average Loss:  25.354715298096647 	Reconstruction Loss: 19.66483805181437 	KL Loss: 5.689877267491403
	Epoch 6 	Average Loss:  24.97050038459218 	Reconstruction Loss: 19.132133463855876 	KL Loss: 5.83836688067462
	Epoch 7 	Average Loss:  24.647955223289696 	Reconstruction Loss: 18.720558252518703 	KL Loss: 5.927396947205297
	Epoch 8 	Average Loss:  24.430987808050798 	Reconstruction Loss: 18.415758061353763 	KL Loss: 6.015229718418195
	Epoch 9 	Average Loss:  24.234366101592663 	Reconstruction Loss: 18.1124643309604

# Training and validation loop

In [27]:
# write your training and validation loop here


tensor([[-9.8135e+00, -7.8513e-02,  8.1249e+00,  2.6417e+01, -7.2266e+03],
        [-1.0521e+01, -7.2719e+00,  1.2743e+01,  1.2470e+00,  1.6353e+05],
        [-1.2339e+01,  1.2947e+05,  1.8986e+01, -7.0713e+00, -6.7309e-01],
        [-2.0914e+01, -1.9231e+00,  1.8045e+01,  4.1455e+00, -2.0132e+00],
        [-1.0656e+01,  5.5100e+00,  1.0418e+01, -1.4071e+00, -3.7740e+04],
        [-9.1636e+00, -5.4948e+03,  2.6198e+00,  2.0623e+00,  5.2142e+01],
        [-1.3785e+01,  2.8850e+00,  8.5814e+00, -1.5732e-01,  2.6206e+01],
        [-1.1000e+01, -4.2588e+00,  1.0319e+01,  1.9233e+00,  5.4276e+02],
        [-3.0660e+01, -5.6718e+00,  2.7112e+01, -1.7166e+00, -5.1036e+00],
        [-5.0666e+00, -1.0215e+01,  5.9489e-01, -3.3136e-01, -1.8082e+02],
        [-1.1114e+01, -4.6019e+01,  3.5699e+00, -3.2093e+00, -1.2595e+01],
        [-1.0067e+01,  9.4589e+02,  4.2327e+00,  2.5071e+00, -9.3977e+03],
        [-4.3183e+00,  9.5311e-01,  5.3914e+00, -4.3793e+00, -2.7470e+00],
        [-1.9431e+01,  6.

In [ ]:
# perform training

# Inspection, Validation, and Analysis

In [4]:
# Inspect, validate, and analyse your trained model